# Main plotting script MATCH2BRAIN -- USED FOR SI Figures!
## i.e., RandNouns & LengthControl BMs

`For main conditions: BETTER: run figure1and2_plotAndStats.py via create_fig1.sh`

## Import packages

In [1]:
save = True

In [2]:
#import packages
import os
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import numpy as np
import pandas as pd
from matplotlib.gridspec import GridSpec
import json

In [3]:
# import from other scripts
import plot_utils
import stats_utils

## Define approach configuration

In [4]:
model_identifier="gpt2-xl"

In [5]:
#whether or not to include control condition random nouns (used for SI)
randomnouns=False
length_control=False

In [6]:
#load configuration
with open('configurations.json', 'r') as f:
    config = json.load(f)
    
list(config.keys())

['TrainIntact-TestPerturbed:contextualized',
 'TrainIntact-TestPerturbed:decontextualized',
 'TrainPerturbed-TestPerturbed:contextualized',
 'TrainPerturbed-TestPerturbed:decontextualized',
 'TrainPerturbed-TestPerturbed:cv-by-passage']

In [7]:
# get setting
approach = "TrainIntact-TestPerturbed:contextualized"

emb_context=config[approach]["emb_context"]
split_coord=config[approach]["split_coord"]
testonperturbed=config[approach]["testonperturbed"]

## Get stimuli

In [8]:
stimuli_df = plot_utils.get_sample_stimuli(randomnouns=randomnouns,length_control=length_control)
stimuli_df

Original | original
Loading lookup from /om2/user/ckauf/anaconda39/envs/perturbed3.8/lib/python3.8/site-packages/brainio_collection/lookup.csv
/net/vast-storage.ib.cluster/scratch/vast/cpl/ckauf/perturbed-neural-nlp/neural_nlp/../ressources/stimuli
I AM USING THE NEW SPLIT FUNCTION
I AM USING THE NEW CROSSVALIDATION SCRIPT
Scr1 | scrambled1
Scr3 | scrambled3
Scr5 | scrambled5
Scr7 | scrambled7
backward | backward
lowPMI | lowpmi
lowPMI_random | lowpmi-random
nouns | nouns
nounsverbs | nounsverbs
nounsverbsadj | nounsverbsadj
contentwords | contentwords
functionwords | functionwords
sentenceshuffle-withinpassage | sent_passage
sentenceshuffle-withintopic | sent_topic
sentenceshuffle-random | sent_random
random | random-wl


,condition,stimulus
0,original,beekeeping encourages the conservation of loca...
1,scrambled1,beekeeping encourages the conservation of habi...
2,scrambled3,beekeeping conservation encourages the of habi...
3,scrambled5,conservation beekeeping encourages the habitat...
4,scrambled7,beekeeping conservation of encourages habitats...
5,backward,habitats local of conservation the encourages ...
6,lowpmi,beekeeping conservation habitats the of encour...
7,lowpmi-random,encourages the beekeeping of habitats conserva...
8,nouns,conservation habitats.
9,nounsverbs,beekeeping encourages conservation habitats.


In [9]:
# function check (this is done in the main function below!)
scores_df = plot_utils.get_best_scores_df(model_identifier=model_identifier,
                                          emb_context=emb_context,
                                          split_coord=split_coord,
                                          testonperturbed=testonperturbed,
                                          randomnouns=randomnouns,length_control=length_control)
scores_df

,score,error,condition,category
0,1.107199,0.141840,original,original
1,1.019374,0.120393,scrambled1,word-order
2,0.924364,0.086006,scrambled3,word-order
3,0.907545,0.071722,scrambled5,word-order
4,0.914475,0.099400,scrambled7,word-order
5,0.835083,0.064965,backward,word-order
6,0.933212,0.070269,lowpmi,word-order
7,0.889217,0.093797,lowpmi-random,word-order
8,0.644818,0.078112,nouns,information-loss
9,0.808808,0.098496,nounsverbs,information-loss


### merge with stimuli df

In [10]:
full_df = scores_df.merge(stimuli_df, on='condition', how='inner')
full_df

,score,error,condition,category,stimulus
0,1.107199,0.141840,original,original,beekeeping encourages the conservation of loca...
1,1.019374,0.120393,scrambled1,word-order,beekeeping encourages the conservation of habi...
2,0.924364,0.086006,scrambled3,word-order,beekeeping conservation encourages the of habi...
3,0.907545,0.071722,scrambled5,word-order,conservation beekeeping encourages the habitat...
4,0.914475,0.099400,scrambled7,word-order,beekeeping conservation of encourages habitats...
5,0.835083,0.064965,backward,word-order,habitats local of conservation the encourages ...
6,0.933212,0.070269,lowpmi,word-order,beekeeping conservation habitats the of encour...
7,0.889217,0.093797,lowpmi-random,word-order,encourages the beekeeping of habitats conserva...
8,0.644818,0.078112,nouns,information-loss,conservation habitats.
9,0.808808,0.098496,nounsverbs,information-loss,beekeeping encourages conservation habitats.


In [11]:
#Rename conditions according to new names:
full_df['condition'] = full_df['condition'].map(plot_utils.COND2LABEL)

In [12]:
full_df["labelname"] = [': '.join(i) for i in zip(full_df["condition"].map(str),full_df["stimulus"])]
full_df

,score,error,condition,category,stimulus,labelname
0,1.107199,0.141840,Original,original,beekeeping encourages the conservation of loca...,Original: beekeeping encourages the conservati...
1,1.019374,0.120393,1LocalWordSwap,word-order,beekeeping encourages the conservation of habi...,1LocalWordSwap: beekeeping encourages the cons...
2,0.924364,0.086006,3LocalWordSwaps,word-order,beekeeping conservation encourages the of habi...,3LocalWordSwaps: beekeeping conservation encou...
3,0.907545,0.071722,5LocalWordSwaps,word-order,conservation beekeeping encourages the habitat...,5LocalWordSwaps: conservation beekeeping encou...
4,0.914475,0.099400,7LocalWordSwaps,word-order,beekeeping conservation of encourages habitats...,7LocalWordSwaps: beekeeping conservation of en...
5,0.835083,0.064965,ReverseOrder,word-order,habitats local of conservation the encourages ...,ReverseOrder: habitats local of conservation t...
6,0.933212,0.070269,LowPMI,word-order,beekeeping conservation habitats the of encour...,LowPMI: beekeeping conservation habitats the o...
7,0.889217,0.093797,LowPMIRand,word-order,encourages the beekeeping of habitats conserva...,LowPMIRand: encourages the beekeeping of habit...
8,0.644818,0.078112,Nouns,information-loss,conservation habitats.,Nouns: conservation habitats.
9,0.808808,0.098496,NounsVerbs,information-loss,beekeeping encourages conservation habitats.,NounsVerbs: beekeeping encourages conservation...


## Get stats

In [13]:
import scipy
import statsmodels

#Benjamini/Hochberg corrected
def get_ttest_results(model_identifier, emb_context="Passage", split_coord="Sentence",
                testonperturbed=False, category=None, randomnouns=False, length_control=False):
    
    subdf = stats_utils.get_stats_df(model_identifier=model_identifier, emb_context=emb_context,
                                     split_coord=split_coord, testonperturbed=testonperturbed,
                                    randomnouns = randomnouns, length_control=length_control)

    pvals2original, pvals2random = [], []
    ttest2original, ttest2random = [], []
    cohensd2original, cohensd2random = [], []
    conds = []
    
    CAT2COND, COND2CAT = plot_utils.get_conditions(testonperturbed=False,
                                                   randomnouns = randomnouns,
                                                   length_control=length_control)
    for cond in CAT2COND[category]:
        if length_control and "random-wl" in cond:
            continue
        
        #adjust names for consistency
        if cond == 'sentenceshuffle_random':
            cond = 'sent_random'
        elif cond == 'sentenceshuffle_passage':
            cond = 'sent_passage'
        elif cond == 'sentenceshuffle_topic':
            cond = 'sent_topic'

        #get subject scores
        original_scores = list(subdf[subdf['condition'] == 'original']["values"])
        cond_scores = list(subdf[subdf['condition'] == cond]["values"])
        random_scores = list(subdf[subdf['condition'] == 'random-wl']["values"])
        
        #get ttest
        ttest2orig, pval2orig = scipy.stats.ttest_ind(original_scores,cond_scores)
        ttest2rand, pval2rand = scipy.stats.ttest_ind(random_scores,cond_scores)
        
        # get effect size
        cohensd2orig = stats_utils.cohens_d(original_scores, cond_scores)
        cohensd2rand = stats_utils.cohens_d(random_scores, cond_scores)
        
        conds.append(cond)
        pvals2original.append(pval2orig)
        ttest2original.append(ttest2orig)
        pvals2random.append(pval2rand)
        ttest2random.append(ttest2rand)
        cohensd2original.append(cohensd2orig)
        cohensd2random.append(cohensd2rand)
        
    # correct for multiple comparisons
    # statsmodels.stats.multitest.fdrcorrection(pvals) = statsmodels.stats.multitest.multipletests(pvals, method='fdr_bh')
    # first output is list of Booleans indicating whether to reject null hypothesis or not
    _, adjusted_pvals2original = statsmodels.stats.multitest.fdrcorrection(pvals2original)
    _, adjusted_pvals2random = statsmodels.stats.multitest.fdrcorrection(pvals2random)
    
    #assign significance levels
    significance2original = stats_utils.assign_significance_labels(adjusted_pvals2original)
    significance2random = stats_utils.assign_significance_labels(adjusted_pvals2random)
    
    
    stats_df = pd.DataFrame({
        "condition": conds,
        "ttest2original" : ttest2original,
        "ttest2random" : ttest2random,
        "adjusted_pvals2original" : adjusted_pvals2original,
        "adjusted_pvals2random" : adjusted_pvals2random,
        "cohensd2original" : cohensd2original,
        "cohensd2random" : cohensd2random,
        "significance2original" : significance2original,
        "significance2random" : significance2random,
        "pvals2original" : pvals2original,
        "pvals2random" : pvals2random
    })
    
    return stats_df

In [14]:
subdf = stats_utils.get_stats_df(model_identifier=model_identifier, emb_context=emb_context,
                                     split_coord=split_coord, testonperturbed=testonperturbed,
                                    randomnouns = randomnouns, length_control=length_control)

random_scores = list(subdf[subdf['condition'] == 'random-wl']["values"])
tstat, pval = scipy.stats.ttest_1samp(random_scores, popmean=0, axis=0, nan_policy='propagate', alternative='two-sided')
print(f"Testing for difference to 0 for random-wl for {model_identifier}")
print(f"One-sample ttest: {tstat}\npvalue: {pval}\n")


random_sent_scores = list(subdf[subdf['condition'] == 'sent_random']["values"])
tstat, pval = scipy.stats.ttest_1samp(random_sent_scores, popmean=0, axis=0, nan_policy='propagate', alternative='two-sided')
print(f"Testing for difference to 0 for sent_random for {model_identifier}")
print(f"One-sample ttest: {tstat}\npvalue: {pval}")

Testing for difference to 0 for random-wl for gpt2-xl
One-sample ttest: 3.1723661545617863
pvalue: 0.011322832097651767

Testing for difference to 0 for sent_random for gpt2-xl
One-sample ttest: 2.6719153879579554
pvalue: 0.02554216696619439


In [15]:
subdf

,condition,values,category
84,original,0.357018,original
82,original,0.308316,original
83,original,0.339043,original
85,original,0.395303,original
86,original,0.397626,original
...,...,...,...
94,random-wl,0.054398,control
93,random-wl,0.018776,control
92,random-wl,0.031188,control
91,random-wl,0.032119,control


In [16]:
length_control_scores = list(subdf[subdf['condition'] == 'length-control']["values"])
tstat, pval = scipy.stats.ttest_1samp(length_control_scores, popmean=0, axis=0, nan_policy='propagate', alternative='two-sided')
print(f"Testing for difference to 0 for length-control for {model_identifier}")
print(f"One-sample ttest: {tstat}\npvalue: {pval}")

Testing for difference to 0 for length-control for gpt2-xl
One-sample ttest: nan
pvalue: nan


In [17]:
if randomnouns:
    subdf = stats_utils.get_stats_df(model_identifier=model_identifier, emb_context=emb_context,
                                     split_coord=split_coord, testonperturbed=testonperturbed,
                                    randomnouns = randomnouns,length_control=length_control)
    
    randomnouns_scores = list(subdf[subdf['condition'] == 'random-nouns']["values"])
    functionword_scores = list(subdf[subdf['condition'] == 'functionwords']["values"])
    #get ttest
    ttest, pval = scipy.stats.ttest_ind(randomnouns_scores, functionword_scores)
    # get effect size
    cohensd = stats_utils.cohens_d(randomnouns_scores, functionword_scores)
    print(f"RANDOM NOUNS vs. FUNCTIONWORDS \nttest: {ttest} \npval: {pval} \ncohen's d: {cohensd}")

In [ ]:
frames = []
if not length_control:
    for category in ["word-order", "information-loss", "semantic-distance"]:
        stats_df = get_ttest_results(model_identifier=model_identifier,
                          emb_context=emb_context,
                          split_coord=split_coord,
                          testonperturbed=testonperturbed,
                          category=category, randomnouns=randomnouns)
        frames.append(stats_df)
else:
    stats_df = get_ttest_results(model_identifier=model_identifier,
                      emb_context=emb_context,
                      split_coord=split_coord,
                      testonperturbed=testonperturbed,
                      category="control", randomnouns=randomnouns,
                                length_control=length_control)
    frames.append(stats_df)
full_stats_df = pd.concat(frames)

full_stats_df

In [ ]:
#Rename conditions according to new names:
full_stats_df['condition'] = full_stats_df['condition'].map(plot_utils.COND2LABEL)
full_stats_df

In [ ]:
if save and length_control:
    full_stats_df.to_csv(f"results_paper/figure1+2/stats_{approach}_length_control.csv",index=False)

In [ ]:
test = full_stats_df[['condition', 'ttest2original', 'adjusted_pvals2original', 'cohensd2original']].T
test, test.columns = test[1:] , test.iloc[0]
test

In [ ]:
full_stats_df[['condition', 'ttest2original', 'adjusted_pvals2original', 'cohensd2original']]

In [ ]:
full_stats_df[['condition', 'ttest2random', 'adjusted_pvals2random', 'cohensd2random']]

## Plot

In [ ]:
#define global figure settings
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['svg.fonttype'] = 'none'

custom_params = {"axes.spines.right": False,
                 "axes.spines.top": False,
                 'ytick.left': True,
                 'xtick.bottom': True,
                'grid.linestyle': "" #gets rid of horizontal lines
                }
sns.set_theme(font_scale=1.4, style="white", rc=custom_params)

In [ ]:
#define colors
CAT2COLOR = {
    "original": "dimgray",
    #
    "word-order": sns.cubehelix_palette(7, start=.2, rot=-.25, dark=0.2, light=.9, reverse=True),
    "information-loss": sns.cubehelix_palette(5, start=2, rot=0, dark=0.2, light=.85, reverse=True),
    #cut off as last gradient color is similar across colors
    "semantic-distance": sns.light_palette("maroon", 4, reverse=True)[:3],
    #
    "control": "lightgray"
}

if randomnouns:
    CAT2COLOR["information-loss"] = sns.cubehelix_palette(6, start=2, rot=0, dark=0.2, light=.85, reverse=True)

In [ ]:
import math

categories = ["word-order", "information-loss", "semantic-distance"]
if randomnouns:
    categories = ["information-loss"]
if length_control:
    categories = ["control"]
    
num_bars = [len(full_df[full_df["category"] == cat]) for cat in categories]
max_bars = max(num_bars)
nrows = len(categories)

fig2 = plt.figure(constrained_layout=True, figsize=(8*nrows, 11), facecolor='white')
props = [1 + 0.2*x for x in num_bars]
spec2 = GridSpec(ncols=nrows, nrows=1, figure=fig2)
f2_ax = []

to_add = 0
    
for ind, current_category in enumerate(categories):
    #set up subplot
    if ind == 0:
        f2_ax.append(fig2.add_subplot(spec2[0, ind]))
    else:
        f2_ax.append(fig2.add_subplot(spec2[0, ind], sharey=f2_ax[0]))

    categories = ["original", current_category, "control"]
    plot_df = full_df[full_df["category"].isin(categories)]
    
    if length_control:
        colors = [CAT2COLOR["original"]] + [CAT2COLOR[current_category]] + [CAT2COLOR["control"]]
    else:
        colors = [CAT2COLOR["original"]] + CAT2COLOR[current_category] + [CAT2COLOR["control"]]
    
    x_pos = np.arange(len(plot_df))
    scores = list(plot_df['score'])
    errors = list(plot_df['error'])
    conditions = list(plot_df['condition'])
                
    stimuli = [list(plot_df.loc[plot_df["category"] == cat]["labelname"]) for cat in categories]
    stimuli = plot_utils.flatten_list(stimuli)
    from textwrap import fill
    stimuli = [fill(l, 75) for l in stimuli]
    
    #add empty slots for same bar width
    if len(x_pos) != max_bars + 2: # +2 because of original and random-wl
        to_add = max_bars - num_bars[ind]
        x_pos = np.arange(len(plot_df) + to_add)
        multiplier1 = math.floor(to_add/2)
        multiplier2 = math.ceil(to_add/2)

        insert_at1 = 1
        insert_at2 = -1

        insert_elements1 = [np.nan]*multiplier1
        insert_elements2 = [np.nan]*multiplier2
        #
        scores[insert_at1:insert_at1] = insert_elements1
        scores[insert_at2:insert_at2] = insert_elements2
        #
        errors[insert_at1:insert_at1] = insert_elements1
        errors[insert_at2:insert_at2] = insert_elements2

        insert_elements1 = [""]*multiplier1
        insert_elements2 = [""]*multiplier2
        #
        conditions[insert_at1:insert_at1] = insert_elements1
        conditions[insert_at2:insert_at2] = insert_elements2
        #
        stimuli[insert_at1:insert_at1] = insert_elements1
        stimuli[insert_at2:insert_at2] = insert_elements2

        insert_elements1 = ["white"]*multiplier1
        insert_elements2 = ["white"]*multiplier2
        
        colors[insert_at1:insert_at1] = insert_elements1
        colors[insert_at2:insert_at2] = insert_elements2

    f2_ax[-1].bar(x_pos, scores,
        yerr=errors,
        align='center',
        alpha=0.9, #color intensity
        ecolor='black',
        capsize=5, #error-bar width
        color=colors)
    
    CAT2COND, COND2CAT = plot_utils.get_conditions(randomnouns=randomnouns, length_control=length_control)
    
    #annotate stats 2 original
    for ind_c, cond in enumerate(CAT2COND[current_category]):
        #adjust names for consistency
        if cond == 'sentenceshuffle_random':
            cond = 'sent_random'
        elif cond == 'sentenceshuffle_passage':
            cond = 'sent_passage'
        elif cond == 'sentenceshuffle_topic':
            cond = 'sent_topic'
            
        cond = plot_utils.COND2LABEL[cond]
            
        if cond in ["Original", "RandWordList"]:
            continue

        idx_random = conditions.index("RandWordList") - to_add

        positions = [i for i, score in enumerate(scores) if not np.isnan(score)]https://www.youtube.com/watch?v=7GjCtCTZKxQ
        print(positions)
        
        #add stats annotations for comparison with original score
        heights = [scores[0]] * len(positions)
        #add height offset to annotations
        height_offset = [0.1 * i for i in range(len(heights))]
        heights = [sum(x) for x in zip(heights, height_offset)]
        label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2original"].item()
        stats_utils.barplot_annotate_brackets(0, ind_c+1, data=label, center=positions, height=heights, fs=10)
        
        #add stats annotations for comparison with random-wl score
        heights = [0] * len(positions)
        #add height offset to annotations
        height_offset = [-0.1 * i for i in range(len(heights))]
        heights = [sum(x) for x in zip(heights, height_offset)]
        label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2random"].item()
        stats_utils.barplot_annotate_brackets(ind_c+1, idx_random, data=label, center=positions, height=heights,
                                             updown="down", fs=10)
    
    #add horizontal lines for original and random-wl
    orig_score = full_df[full_df["condition"]=="Original"]["score"].item()
    random_score = full_df[full_df["condition"]=="RandWordList"]["score"].item()
    f2_ax[-1].axhline(y=orig_score, color=CAT2COLOR["original"], linestyle=':', dashes=(5, 3), linewidth=1)
    f2_ax[-1].axhline(y=random_score, color=CAT2COLOR["control"], linestyle=":", dashes=(5, 3), linewidth=1)
    
    f2_ax[-1].axhline(y=0, color="black")

    # add legend (sample stimuli)
    # map names to colors
    cmap = dict(zip(conditions, colors))
    cmap = {k: v for k, v in cmap.items() if k}
    # create the rectangles for the legend
    from matplotlib.patches import Patch
    patches = [Patch(color=v, label=k, alpha=0.8) for k, v in cmap.items()]
    # remove empty strings from labels
    stimuli = [x for x in stimuli if x]
    # add the legend
    f2_ax[-1].legend(title='', labels=stimuli, handles=patches,
                     loc='upper center', bbox_to_anchor=(0.5, -0.4),
                     title_fontsize=15, prop={'size': 13.5})
    
    
    #f2_ax[-1].set_ylim([-0.1 + -0.1*num_bars[ind], 1.25 + 0.1*num_bars[ind]])
    
    # TICKS
    ## to get current ones: f2_ax[-1].get_yticks()
    if ind == 0:
        yticks = [0, 0.25, 0.5, 0.75, 1]
        f2_ax[-1].set_yticks(yticks)
    #set yticks
    f2_ax[-1].set_xticks(positions)
    xticknames = ["\nFrom".join(elm.split("From")) for elm in conditions]
    f2_ax[-1].set_xticklabels([x for x in xticknames if x], rotation=70, ha="right", rotation_mode="anchor")
    
    f2_ax[-1].set_ylabel('Normalized predictivity')
    
    f2_ax[-1].set_title(f"{current_category} manipulations",pad=0.5)

    plt.tight_layout()

save=True
if save:
    if not (randomnouns or length_control):
        plt.savefig(f'results_paper/figure1+2/{approach}.svg', dpi=180)
        plt.savefig(f'results_paper/figure1+2/{approach}.png', dpi=180, bbox_inches='tight')

In [ ]:
#VERTICAL STACKING
if save and (approach == "TrainIntact-TestPerturbed:contextualized" or length_control):
    categories = ["word-order", "information-loss", "semantic-distance"]
    if randomnouns:
        categories = ["information-loss"]
    if length_control:
        categories = ["control"]
    num_bars = [len(full_df[full_df["category"] == cat]) for cat in categories]
    max_bars = max(num_bars)
    nrows = len(categories)

    if length_control:
        fig2 = plt.figure(constrained_layout=True, figsize=(12, 5*nrows), facecolor='white')
    else:
        fig2 = plt.figure(constrained_layout=True, figsize=(15, 7*nrows), facecolor='white')
    props = [1 + 0.2*x for x in num_bars]
    spec2 = GridSpec(ncols=1, nrows=nrows, figure=fig2, height_ratios=props)
    f2_ax = []

    to_add = 0

    for ind, current_category in enumerate(categories):
        #set up subplot
        f2_ax.append(fig2.add_subplot(spec2[ind, 0]))

        categories = ["original", current_category, "control"]
        plot_df = full_df[full_df["category"].isin(categories)]

        if length_control:
            colors = [CAT2COLOR["original"]] + [CAT2COLOR[current_category]] + [CAT2COLOR["control"]]
        else:
            colors = [CAT2COLOR["original"]] + CAT2COLOR[current_category] + [CAT2COLOR["control"]]

        x_pos = np.arange(len(plot_df))
        scores = list(plot_df['score'])
        errors = list(plot_df['error'])
        conditions = list(plot_df['condition'])

        stimuli = [list(plot_df.loc[plot_df["category"] == cat]["labelname"]) for cat in categories]
        stimuli = plot_utils.flatten_list(stimuli)
        from textwrap import fill
        stimuli = [fill(l, 75) for l in stimuli]

        #add empty slots for same bar width
        if len(x_pos) != max_bars + 2: # +2 because of original and random-wl
            to_add = max_bars - num_bars[ind]
            x_pos = np.arange(len(plot_df) + to_add)
            multiplier = int(to_add/2)

            insert_at1 = 1
            insert_at2 = -1

            insert_elements = [np.nan]*multiplier
            #
            scores[insert_at1:insert_at1] = insert_elements
            scores[insert_at2:insert_at2] = insert_elements
            #
            errors[insert_at1:insert_at1] = insert_elements
            errors[insert_at2:insert_at2] = insert_elements

            insert_elements = [""]*multiplier
            #
            conditions[insert_at1:insert_at1] = insert_elements
            conditions[insert_at2:insert_at2] = insert_elements
            #
            stimuli[insert_at1:insert_at1] = insert_elements
            stimuli[insert_at2:insert_at2] = insert_elements

            insert_elements = ["white"]*multiplier
            colors[insert_at1:insert_at1] = insert_elements
            colors[insert_at2:insert_at2] = insert_elements

        f2_ax[-1].bar(x_pos, scores,
            yerr=errors,
            align='center',
            alpha=0.9, #color intensity
            ecolor='black',
            capsize=5, #error-bar width
            color=colors)

        #annotate stats 2 original
        for ind_c, cond in enumerate(CAT2COND[current_category]):
            #adjust names for consistency
            if cond == 'sentenceshuffle_random':
                cond = 'sent_random'
            elif cond == 'sentenceshuffle_passage':
                cond = 'sent_passage'
            elif cond == 'sentenceshuffle_topic':
                cond = 'sent_topic'

            cond = plot_utils.COND2LABEL[cond]
            
            if cond in ["Original", "RandWordList"]:
                continue

            idx_random = conditions.index("RandWordList") - to_add

            positions = [i for i, score in enumerate(scores) if not np.isnan(score)]

            #add stats annotations for comparison with original score
            heights = [scores[0]] * len(positions)
            #add height offset to annotations
            height_offset = [0.1 * i for i in range(len(heights))]
            heights = [sum(x) for x in zip(heights, height_offset)]
            label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2original"].item()
            stats_utils.barplot_annotate_brackets(0, ind_c+1, data=label, center=positions, height=heights, fs=10)

            #add stats annotations for comparison with random-wl score
            heights = [0] * len(positions)
            #add height offset to annotations
            height_offset = [-0.1 * i for i in range(len(heights))]
            heights = [sum(x) for x in zip(heights, height_offset)]
            label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2random"].item()
            stats_utils.barplot_annotate_brackets(ind_c+1, idx_random, data=label, center=positions, height=heights,
                                                 updown="down", fs=10)

        #add horizontal lines for original and random-wl
        orig_score = full_df[full_df["condition"]=="Original"]["score"].item()
        random_score = full_df[full_df["condition"]=="RandWordList"]["score"].item()
        f2_ax[-1].axhline(y=orig_score, color=CAT2COLOR["original"], linestyle=':', dashes=(5, 3), linewidth=1)
        f2_ax[-1].axhline(y=random_score, color=CAT2COLOR["control"], linestyle=":", dashes=(5, 3), linewidth=1)

        f2_ax[-1].axhline(y=0, color="black")

        # add legend (sample stimuli)
        # map names to colors
        cmap = dict(zip(conditions, colors))
        cmap = {k: v for k, v in cmap.items() if k}
        # create the rectangles for the legend
        from matplotlib.patches import Patch
        patches = [Patch(color=v, label=k, alpha=0.8) for k, v in cmap.items()]
        # remove empty strings from labels
        stimuli = [x for x in stimuli if x]
        # add the legend
        if length_control:
            f2_ax[-1].legend(title=f'length-control manipulation', labels=stimuli, handles=patches, bbox_to_anchor=(1.05, 0.5),
                     loc='center left', title_fontsize=15, prop={'size': 13.5})
        else:
            f2_ax[-1].legend(title=f'{current_category} manipulations', labels=stimuli, handles=patches, bbox_to_anchor=(1.05, 0.5),
                     loc='center left', title_fontsize=15, prop={'size': 13.5})            


        f2_ax[-1].set_ylim([-0.1 + -0.1*num_bars[ind], 1.25 + 0.1*num_bars[ind]])

        # TICKS
        ## to get current ones: f2_ax[-1].get_yticks()
        yticks = [0, 0.25, 0.5, 0.75, 1]
        f2_ax[-1].set_yticks(yticks)
        #set yticks
        f2_ax[-1].set_xticks(positions)
        xticknames = ["\nFrom".join(elm.split("From")) for elm in conditions]
        f2_ax[-1].set_xticklabels([x for x in xticknames if x], rotation=70, ha="right", rotation_mode="anchor")

        f2_ax[-1].set_ylabel('Normalized predictivity')

        plt.tight_layout()
        if length_control:
            plt.savefig(f'results_paper/figure1+2/approach={approach}_length_control.svg', dpi=300)
            plt.savefig(f'results_paper/figure1+2/approach={approach}_length_control.png', dpi=300)
        else:
            plt.savefig('results_paper/figure1+2/main.svg', dpi=300)
            plt.savefig('results_paper/figure1+2/main.png', dpi=300)           
            
            

In [ ]:
break

In [ ]:
vertical=False

In [ ]:
categories = ["word-order", "information-loss", "semantic-distance"]
if randomnouns:
    categories = ["information-loss"]
    
num_bars = [len(full_df[full_df["category"] == cat]) for cat in categories]
max_bars = max(num_bars)
props = [1 + 0.2*x for x in num_bars]

if vertical:
    nrows = len(categories)
    figsize=(15, 7*nrows)
    ncols = 1
    height_ratios=props
else:
    nrows = 1
    figsize=(8*len(categories), 11)
    ncols = len(categories)
    height_ratios=None

fig2 = plt.figure(constrained_layout=True, figsize=figsize, facecolor='white')
spec2 = GridSpec(ncols=ncols, nrows=nrows, figure=fig2, height_ratios=height_ratios)
f2_ax = []

to_add = 0
    
for ind, current_category in enumerate(categories):
    #set up subplot
    if vertical:
        f2_ax.append(fig2.add_subplot(spec2[ind, 0]))
    else:
        if ind == 0:
            f2_ax.append(fig2.add_subplot(spec2[0, ind]))
        else:
            f2_ax.append(fig2.add_subplot(spec2[0, ind], sharey=f2_ax[0]))

    categories = ["original", current_category, "control"]
    plot_df = full_df[full_df["category"].isin(categories)]
    
    colors = [CAT2COLOR["original"]] + CAT2COLOR[current_category] + [CAT2COLOR["control"]]
    
    x_pos = np.arange(len(plot_df))
    scores = list(plot_df['score'])
    errors = list(plot_df['error'])
    conditions = list(plot_df['condition'])
                
    stimuli = [list(plot_df.loc[plot_df["category"] == cat]["labelname"]) for cat in categories]
    stimuli = plot_utils.flatten_list(stimuli)
    from textwrap import fill
    stimuli = [fill(l, 75) for l in stimuli]
    
    #add empty slots for same bar width
    if len(x_pos) != max_bars + 2: # +2 because of original and random-wl
        to_add = max_bars - num_bars[ind]
        x_pos = np.arange(len(plot_df) + to_add)
        multiplier1 = math.floor(to_add/2)
        multiplier2 = math.ceil(to_add/2)

        insert_at1 = 1
        insert_at2 = -1

        insert_elements1 = [np.nan]*multiplier1
        insert_elements2 = [np.nan]*multiplier2
        #
        scores[insert_at1:insert_at1] = insert_elements1
        scores[insert_at2:insert_at2] = insert_elements2
        #
        errors[insert_at1:insert_at1] = insert_elements1
        errors[insert_at2:insert_at2] = insert_elements2

        insert_elements1 = [""]*multiplier1
        insert_elements2 = [""]*multiplier2
        #
        conditions[insert_at1:insert_at1] = insert_elements1
        conditions[insert_at2:insert_at2] = insert_elements2
        #
        stimuli[insert_at1:insert_at1] = insert_elements1
        stimuli[insert_at2:insert_at2] = insert_elements2

        insert_elements1 = ["white"]*multiplier1
        insert_elements2 = ["white"]*multiplier2
        
        colors[insert_at1:insert_at1] = insert_elements1
        colors[insert_at2:insert_at2] = insert_elements2

    f2_ax[-1].bar(x_pos, scores,
        yerr=errors,
        align='center',
        alpha=0.9, #color intensity
        ecolor='black',
        capsize=5, #error-bar width
        color=colors)
    
    CAT2COND, COND2CAT = plot_utils.get_conditions()
    
    #annotate stats 2 original
    for ind_c, cond in enumerate(CAT2COND[current_category]):
        #adjust names for consistency
        if cond == 'sentenceshuffle_random':
            cond = 'sent_random'
        elif cond == 'sentenceshuffle_passage':
            cond = 'sent_passage'
        elif cond == 'sentenceshuffle_topic':
            cond = 'sent_topic'
            
        cond = plot_utils.COND2LABEL[cond]
            
        if cond in ["Original", "RandWordList"]:
            continue

        idx_random = conditions.index("RandWordList") - to_add

        positions = [i for i, score in enumerate(scores) if not np.isnan(score)]
        
        #add stats annotations for comparison with original score
        heights = [scores[0]] * len(positions)
        #add height offset to annotations
        height_offset = [0.1 * i for i in range(len(heights))]
        heights = [sum(x) for x in zip(heights, height_offset)]
        label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2original"].item()
        stats_utils.barplot_annotate_brackets(0, ind_c+1, data=label, center=positions, height=heights, fs=10)
        
        #add stats annotations for comparison with random-wl score
        heights = [0] * len(positions)
        #add height offset to annotations
        height_offset = [-0.1 * i for i in range(len(heights))]
        heights = [sum(x) for x in zip(heights, height_offset)]
        label = full_stats_df.loc[full_stats_df["condition"] == cond]["significance2random"].item()
        stats_utils.barplot_annotate_brackets(ind_c+1, idx_random, data=label, center=positions, height=heights,
                                             updown="down", fs=10)
    
    #add horizontal lines for original and random-wl
    orig_score = full_df[full_df["category"]=="original"]["score"].item()
    random_score = full_df[full_df["category"]=="control"]["score"].item()
    f2_ax[-1].axhline(y=orig_score, color=CAT2COLOR["original"], linestyle=':', dashes=(5, 3), linewidth=1)
    f2_ax[-1].axhline(y=random_score, color=CAT2COLOR["control"], linestyle=":", dashes=(5, 3), linewidth=1)
    
    f2_ax[-1].axhline(y=0, color="black")

    # add legend (sample stimuli)
    # map names to colors
    cmap = dict(zip(conditions, colors))
    cmap = {k: v for k, v in cmap.items() if k}
    # create the rectangles for the legend
    from matplotlib.patches import Patch
    patches = [Patch(color=v, label=k, alpha=0.8) for k, v in cmap.items()]
    # remove empty strings from labels
    stimuli = [x for x in stimuli if x]
    
    # add the legend
    if vertical:
        f2_ax[-1].legend(title=f'{current_category} manipulations', labels=stimuli, handles=patches,
                         bbox_to_anchor=(1.05, 0.5), loc='center left', title_fontsize=15, prop={'size': 13.5})
        f2_ax[-1].set_ylim([-0.1 + -0.1*num_bars[ind], 1.25 + 0.1*num_bars[ind]])
    else:
        f2_ax[-1].legend(title='', labels=stimuli, handles=patches,
                         loc='upper center', bbox_to_anchor=(0.5, -0.4),
                         title_fontsize=15, prop={'size': 13.5})
    
    
    # TICKS
    ## to get current ones: f2_ax[-1].get_yticks()
    if ind == 0:
        yticks = [0, 0.25, 0.5, 0.75, 1]
        f2_ax[-1].set_yticks(yticks)
    #set yticks
    f2_ax[-1].set_xticks(positions)
    xticknames = ["\nFrom".join(elm.split("From")) for elm in conditions]
    f2_ax[-1].set_xticklabels([x for x in xticknames if x], rotation=70, ha="right", rotation_mode="anchor")
    
    f2_ax[-1].set_ylabel('Normalized predictivity')
    plt.tight_layout()
    if not vertical:
        f2_ax[-1].set_title(f"{current_category} manipulations",pad=0.5)

# Within manipulation category stats

In [ ]:
stats_df = stats_utils.get_stats_df(model_identifier=model_identifier, emb_context=emb_context,
                                     split_coord=split_coord, testonperturbed=testonperturbed,
                                    randomnouns = randomnouns)

#Rename conditions according to new names:
stats_df['condition'] = stats_df['condition'].map(plot_utils.COND2LABEL)
stats_df

In [ ]:
# Plot arguments
heatmap_args = {'linewidths': 0.25, 'linecolor': '0.5', 'clip_on': False, 'square': True, 'cbar_ax_bbox': [0.95, 0.55, 0.04, 0.3]}

In [ ]:
import scikit_posthocs as sp

CAT2COND, COND2CAT = plot_utils.get_conditions()
categories = [x for x in CAT2COND.keys() if x not in ['original', 'control']]

for category in categories:
    print(category)
    sub_df = stats_df.loc[stats_df["category"] == category]
    print(len(sub_df))
    ttest = sp.posthoc_ttest(sub_df, val_col='values', group_col='condition', p_adjust='fdr_bh')
    print(ttest)
        
    plt.figure(figsize=(10,10))
    sp.sign_plot(ttest, annot=ttest, fmt='.3f', **heatmap_args)
    plt.subplots_adjust(top=0.8) 
    plt.title(f'{category}')
    
    ttest_df = pd.DataFrame(ttest)
    
    ttest_df["manipulation"] = [f"{category}"] * len(ttest_df)
    ttest_df.to_csv(f'results_paper/figure1+2/stats_{approach}_within_condition={category}.csv')

    plt.show()

In [ ]:
break

### 3. Define plot function

In [ ]:
main_plotly("gpt2", testonperturbed=True)

In [ ]:
main_plotly("gpt2")

In [ ]:
def main_plotly(model_identifier, emb_context="Passage", split_coord="Sentence", testonperturbed=False):
    scores_df = plot_utils.get_best_scores_df(model_identifier, emb_context=emb_context,
                                      split_coord=split_coord, testonperturbed=testonperturbed)
    barplot_plotly(model_identifier, scores_df, emb_context=emb_context,
            split_coord=split_coord, testonperturbed=testonperturbed)

In [ ]:
# barplot("distilgpt2", scores_df)

In [ ]:
def barplot_plotly(model_identifier, scores_df, emb_context="Passage", split_coord="Sentence", testonperturbed=False):
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    colors = sns.color_palette("rocket")[1:]

    #get proportions for subplot width
    categories = ["original", "control", "information_loss", "meaning", "word_order"]
    props = [len(scores_df.loc[scores_df["category"] == cat])/len(scores_df) for cat in categories]

    #make subplot
    fig = make_subplots(rows=1, cols=len(categories), column_widths=props, shared_yaxes=True, subplot_titles = categories)

    for ind, key in enumerate(categories):
        #subset df
        curr_df = scores_df.loc[scores_df["category"] == key]
        curr_color = colors[ind]
        #add figure trace
        fig.add_trace(
            go.Bar(name=key, x=list(curr_df["condition"]), y=list(curr_df["score"]),
#             marker_color=curr_color,
            error_y={
                "type": "data",
                "array": list(curr_df["error"])
            }
        ),row=1, col=ind+1)

        #uodate layout
    title = f"{model_identifier}"
    if not testonperturbed:
        title += f" | Trained and tested on perturbed"
    fig.update_layout(title_text=f"{model_identifier}",
                      showlegend=False) 
    fig.update_xaxes(tickangle=270)

    if save:
        if not os.path.exists("images"):
            os.mkdir("images")
        fig.write_image(f"images/newest_{title}.png")

    fig.show()

In [ ]:
save = False

In [ ]:
# !pip install -U kaleido

# PLOTLY EXPRESS

In [ ]:
colors = sns.color_palette("rocket")[1:]
sns.catplot(data=scores_df, x='condition', y='score', hue='model', col='category', kind='bar', sharex=False)

In [ ]:
scores_df = main_multi(plotting_models, testonperturbed=True)

In [ ]:
plotting_models = ["distilgpt2", "distilgpt2-avgtoken"]
scores_df = main_multi(plotting_models)

In [ ]:
plt1.set_title("A")
plt2.set_title("B")

full_plt = plt1/plt2
full_plt.savefig("images/newest_resultRobustness_gpt2versions.png")
full_plt.show()

In [ ]:
#!pip install patchworklib

In [ ]:
plt2 = main_multi(plotting_models, testonperturbed=True)

In [ ]:
plt1 = main_multi(plotting_models)

In [ ]:
plotting_models = ["distilgpt2", "gpt2", "gpt2-xl"]

In [ ]:
import pandas as pd

def main_multi(list_of_model_identifiers, emb_context="Passage", split_coord="Sentence", testonperturbed=False):
    frames = []

    for model in list_of_model_identifiers:
        m_scores_df = plot_utils.get_best_scores_df(model, emb_context=emb_context,
                                    split_coord=split_coord, testonperturbed=testonperturbed)
        m_scores_df["model"] = [model] * len(m_scores_df)
        frames.append(m_scores_df)
    
    scores_df = pd.concat(frames)

    if MAD_corrected:
        scores_df["error"] = scores_df["error"].div(1.4826) #MAD is wrong
    
    barplot_multi(list_of_model_identifiers, scores_df, emb_context=emb_context,
            split_coord=split_coord, testonperturbed=testonperturbed)
    
    return plt

In [ ]:
save=True

def barplot_multi(list_of_model_identifiers, scores_df, emb_context="Passage", split_coord="Sentence", testonperturbed=False):

    colors = sns.color_palette("rocket")[1:]
    sns.set(font_scale=1.4)

    #get proportions for subplot width
    categories = ["original", "controls", "information-loss", "sentence-meaning", "word-order"]
    props = [len(scores_df.loc[scores_df["category"] == cat])/len(scores_df) for cat in categories]

    g = sns.catplot(data=scores_df, x='condition', y='score', hue='model', col='category', kind='bar', sharex=False,
                    facet_kws={'gridspec_kws': {"width_ratios": props}}, height=7, aspect=.45, legend_out=True)
 
          
   # g.map(plt.errorbar, "condition", "score", "error", ls='none', color="k", capsize=5)
    g.set_titles(col_template="{col_name}")
    g.set_xlabels("")
    g.set_xticklabels(rotation=80)
    
    if not "gpt2-xl" in list_of_model_identifiers:
        g.set(ylim=(0, 1))
    else:
        g.set(ylim=(0, 1.25))
    
        title_addition = " | decontextualized"
        
    sns.move_legend(g, "upper left", bbox_to_anchor=(1, 0.75))
    g.fig.tight_layout()

    if save:
        g.savefig(f'images/newest_match-to-brain_allgpts.png', bbox_inches='tight', dpi=240)

    plt.show()
    return g.fig

# All models barplot

In [ ]:
main("distilgpt2-avgtoken", testonperturbed=True)

In [ ]:
main("distilgpt2-avgtoken")

In [ ]:
main("distilgpt2", emb_context='Sentence')